In [19]:
import pandas as pd
import pdfplumber

In [20]:
#pip install pdfplumber

In [21]:
import pathlib
pathlib.Path().resolve()

PosixPath('/Users/ytsang/Desktop/Sanction List')

In [22]:
import sys

Path = os.getcwd()

In [23]:
def stringRefining(testString):
 
    output = testString[:8] + testString[13:]
    output = output.split(" ")

    for i in range(len(output)):
        output[i] = output[i].lstrip()

        if "1:" in output[i]:
            output[i] = output[i].replace("1:", '')
        if "2:" in output[i]:
            output[i] = output[i].replace("2:", '')
        if "3:" in output[i]:
            output[i] = output[i].replace("3:", '')
        if "4:" in output[i]:
            output[i] = output[i].replace("4:", '')
        if "5:" in output[i]:
            output[i] = output[i].replace("5:", '')
        if "6:" in output[i]:
            output[i] = output[i].replace("6:", '')
        if ":" in output[i]:
            output[i] = output[i].replace(":", '')
        if "na" in output[i]:
            output[i] = output[i].replace("na", '')

    output = list(filter(None, output))
    return ' '.join(output)

In [24]:
import re
import glob, os

for fileName in glob.glob("*.pdf"):
    if fileName.endswith(".pdf"):
        stinfo = os.stat(fileName)
        last_mtime = stinfo.st_mtime
        last_atime = stinfo.st_atime

        page_count = 0
        page=''
        current_page = ''
        content = []

        #page_number = 3
        idx = 0

        #Path = '/Users/ytsang/Desktop/Sanction List/'
        #fileName = 'HKICPA_UN Consolidated list_20200128.pdf'
        #fileName = input("Enter your file name to be converted: ")

        with pdfplumber.open(Path + '/' + fileName) as pdf:
            page_number = len(pdf.pages)
            while page_count < page_number:
                current_page = pdf.pages[page_count].extract_text()
                page = page + current_page

                #lines = current_page.readlines()
                lines = current_page.split('\n')

                for line in lines:
            
                    if "Name:" in line:
                        if "(original script)" not in line: 
                            content.insert(idx, line)
                            idx +=1

                page_count = page_count + 1


        sanctionList = pd.DataFrame()

        for i in range(len(content)):
            content[i] = stringRefining(content[i])
            sanctionList = sanctionList.append({'Code': content[i][:7], 'Name': content[i][8:]}, ignore_index = True)

        output_filename = fileName[:-4] +'.csv'
        sanctionList.to_csv(output_filename, index=False)

        last_mtime += 100
        last_atime += 100

        os.utime(output_filename, (last_atime, last_mtime))
